https://www.kaggle.com/laknath/blend-it-all

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import xgboost as xgb
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
# cleaned text files. Refer notebook "Preprocessing Text"
train_file = 'train_cleaned.csv' #train.csv
test_file = 'test_cleaned.csv' #test.csv

train = pd.read_csv('../input/' + train_file).fillna(' ')#.sample(1000)
test = pd.read_csv('../input/' + test_file).fillna(' ')#.sample(1000)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

train = train.loc[:,class_names]

### Finding corelation

In [15]:
fname = '../submissions/ensemble/submissions.txt'

with open(fname) as f:
    for line in f:
        print(line)

fileName                                                                                                                                                         date                 description                                                                                                                                                                  status    publicScore  privateScore  

---------------------------------------------------------------------------------------------------------------------------------------------------------------  -------------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------  --------  -----------  ------------  

gru_glove_128_95p_reg.10fold_avg.csv                                                                                                                             2018-03-18 16:57:22  gru_glove_128_95p_reg.10 fold_

In [56]:
#! ls -la '../submissions/ensemble'
fold1 = '../submissions/ensemble/'

submissions = [
    'blend_it_all.gru_glove.fasttext.9868.csv',
    'blend_it_all.9867.csv',
    'dpcnn_glove_32_95p_reg.20fold_avg.9835.csv',
    'gru_glove_128_95p_reg.10fold_avg.9853.csv',    
    'gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv',    
    'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv',
    'lstm_fasttext_256_95p_reg.5fold_avg.9841.csv',
    'cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv',
    'cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_2_fasttext_maxpool_ep2_batch_128_9834.csv',
    'lstm_l1_256_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep2_batch_128_nadam_9834.csv',
    'cnn_2_window_2_3_filter_64_l1_lstm80_spatial_dr_0_4_lstm_dr_0_2_fasttext_ep3_batch_128_9833.csv',
    'gru_l1_gru128_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep1_batch_128_9832.csv',
    'lstm_64_biLSTM_timedistributed_fasttext_pooling_batch32.990567.9831.csv',
    
    'cnn_2_window_2_3_filter_64_l1_lstm80_spatial_dr_0_4_lstm_dr_0_2_dense_64_dr_0_2_glove_ep3_batch_128_9830.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_fasttext_ep4_batch_256_nadam_001_9833.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_glove_ep5_batch_128_nadam_001_9827.csv',
    'toxic_lr_char_tdidf.9792.csv'
]

len(submissions)

17

In [50]:
#! ls -la '../submissions/ensemble'
fold2 = '../submissions/ensemble/downloads/'

submissions = [
    'bi-gru-cnn.9850.csv', 
    'hight_of_blending.9835.csv', 
    'lvl0_wordbatch_clean_oof.9805.csv', 
    'pooled-gru-fasttext.9833.csv', 
    'superblend_1.csv', 
    'bi-lstm-cnn.9841.csv', 
    'lvl0_lgbm_clean_oof.9792.csv', 
    'lvl0_wordbatch_clean_sub.9805.csv', 
    'pooled_gru_glove_r.9815.csv', 
    'tidy_xgb_glm.9784.csv', 
    'corr_blend.9855.csv', 
    'lvl0_lgbm_clean_sub.9792.csv', 
    'oof-stacking-regime.9858.csv', 
    'pooled-gru-with-preprocessing.csv', 
    'toxic-avenger.9823.csv'
]    

len(submissions)

15

In [66]:
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50827

import pandas as pd
import sys
from scipy.stats import ks_2samp

#first_file = fold1+"gru_glove_128_95p_reg.10fold_avg.9853.csv"
#second_file = fold2+"pooled-gru-with-preprocessing.9823.csv"

first_file = fold2+"bi-lstm-cnn.9841.csv" #bi-lstm-cnn.9841.csv
second_file = fold1+"cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv"
print(first_file, second_file)


def corr(first_file, second_file):
    # assuming first column is `class_name_id`
    first_df = pd.read_csv(first_file, index_col=0)
    second_df = pd.read_csv(second_file, index_col=0)
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for class_name in class_names:
        # all correlations
        print('\n Class: %s' % class_name)
        print(' Pearson\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='pearson'))
        print(' Kendall\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='kendall'))
        print(' Spearman\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='spearman'))
        ks_stat, p_value = ks_2samp(first_df[class_name].values,
                                    second_df[class_name].values)
        print(' Kolmogorov-Smirnov test:    KS-stat = %.6f    p-value = %.3e\n'
              % (ks_stat, p_value))

corr(first_file, second_file)

../submissions/ensemble/dpcnn_glove_32_95p_reg.20fold_avg.9835.csv ../submissions/ensemble/cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv

 Class: toxic
 Pearson's correlation score: 0.963573
 Kendall's correlation score: 0.766762
 Spearman's correlation score: 0.920296
 Kolmogorov-Smirnov test:    KS-stat = 0.291537    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.931336
 Kendall's correlation score: 0.695206
 Spearman's correlation score: 0.870392
 Kolmogorov-Smirnov test:    KS-stat = 0.574450    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.973185
 Kendall's correlation score: 0.736112
 Spearman's correlation score: 0.895915
 Kolmogorov-Smirnov test:    KS-stat = 0.375382    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.746064
 Kendall's correlation score: 0.636814
 Spearman's correlation score: 0.832712
 Kolmogorov-Smirnov test:    KS-stat = 0.454304    p-val

In [29]:
# compare all against all
for f1 in submissions:
    for f2 in submissions:
        if f1 != f2:
            print("Corelation between : ", f1, f2)
            corr(fold1+f1, fold2+f2)

Corelation between :  bi-gru-cnn.9850.csv hight_of_blending.9835.csv

 Class: toxic
 Pearson's correlation score: 0.963478
 Kendall's correlation score: 0.771129
 Spearman's correlation score: 0.922691
 Kolmogorov-Smirnov test:    KS-stat = 0.273994    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.876528
 Kendall's correlation score: 0.747843
 Spearman's correlation score: 0.909366
 Kolmogorov-Smirnov test:    KS-stat = 0.554393    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.967480
 Kendall's correlation score: 0.722103
 Spearman's correlation score: 0.882967
 Kolmogorov-Smirnov test:    KS-stat = 0.506379    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.863117
 Kendall's correlation score: 0.660296
 Spearman's correlation score: 0.854674
 Kolmogorov-Smirnov test:    KS-stat = 0.451731    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.942971
 Kendall's correlation score: 0.731355
 Spearm

 Kendall's correlation score: 0.189534
 Spearman's correlation score: 0.279210
 Kolmogorov-Smirnov test:    KS-stat = 0.997748    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.893053
 Kendall's correlation score: 0.574190
 Spearman's correlation score: 0.757862
 Kolmogorov-Smirnov test:    KS-stat = 0.891561    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.774666
 Kendall's correlation score: 0.357714
 Spearman's correlation score: 0.511576
 Kolmogorov-Smirnov test:    KS-stat = 0.982150    p-value = 0.000e+00

Corelation between :  bi-gru-cnn.9850.csv pooled_gru_glove_r.9815.csv

 Class: toxic
 Pearson's correlation score: 0.975404
 Kendall's correlation score: 0.824082
 Spearman's correlation score: 0.953262
 Kolmogorov-Smirnov test:    KS-stat = 0.043679    p-value = 1.929e-127


 Class: severe_toxic
 Pearson's correlation score: 0.905302
 Kendall's correlation score: 0.816104
 Spearman's correlation score: 0.952131
 Kolmogorov-Sm

 Kolmogorov-Smirnov test:    KS-stat = 0.106983    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.883231
 Kendall's correlation score: 0.643123
 Spearman's correlation score: 0.833864
 Kolmogorov-Smirnov test:    KS-stat = 0.156212    p-value = 0.000e+00

Corelation between :  bi-gru-cnn.9850.csv toxic-avenger.9823.csv

 Class: toxic
 Pearson's correlation score: 0.959373
 Kendall's correlation score: 0.759811
 Spearman's correlation score: 0.917069
 Kolmogorov-Smirnov test:    KS-stat = 0.279230    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.846875
 Kendall's correlation score: 0.710863
 Spearman's correlation score: 0.884929
 Kolmogorov-Smirnov test:    KS-stat = 0.531313    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.962782
 Kendall's correlation score: 0.710362
 Spearman's correlation score: 0.876730
 Kolmogorov-Smirnov test:    KS-stat = 0.500261    p-value = 0.000e+00


 Class: threat
 Pearson's


 Class: toxic
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.904156    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.990004    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.947052    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.997004    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.950636    p-value = 0.000e+00


 Class: identity_hate
 Pearson's corre

 Kendall's correlation score: 0.627841
 Spearman's correlation score: 0.825811
 Kolmogorov-Smirnov test:    KS-stat = 0.527957    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.967852
 Kendall's correlation score: 0.696149
 Spearman's correlation score: 0.863761
 Kolmogorov-Smirnov test:    KS-stat = 0.530072    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.907605
 Kendall's correlation score: 0.645215
 Spearman's correlation score: 0.830712
 Kolmogorov-Smirnov test:    KS-stat = 0.546858    p-value = 0.000e+00

Corelation between :  hight_of_blending.9835.csv pooled-gru-with-preprocessing.csv

 Class: toxic
 Pearson's correlation score: 0.962060
 Kendall's correlation score: 0.754885
 Spearman's correlation score: 0.911594
 Kolmogorov-Smirnov test:    KS-stat = 0.225164    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.912141
 Kendall's correlation score: 0.608058
 Spearman's correlation score: 0.791446
 K

...

 Kolmogorov-Smirnov test:    KS-stat = 0.335595    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.873400
 Kendall's correlation score: 0.639474
 Spearman's correlation score: 0.828845
 Kolmogorov-Smirnov test:    KS-stat = 0.340054    p-value = 0.000e+00

Corelation between :  pooled-gru-with-preprocessing.csv lvl0_wordbatch_clean_oof.9805.csv

 Class: toxic
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.904156    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.990004    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: nan
 Kendall's correlation score: nan
 Spearman's correlation score: nan
 Kolmogorov-Smirnov test:    KS-stat = 0.947052    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 

 Spearman's correlation score: 0.829381
 Kolmogorov-Smirnov test:    KS-stat = 0.139915    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.948931
 Kendall's correlation score: 0.695251
 Spearman's correlation score: 0.865578
 Kolmogorov-Smirnov test:    KS-stat = 0.097373    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.845392
 Kendall's correlation score: 0.549517
 Spearman's correlation score: 0.752037
 Kolmogorov-Smirnov test:    KS-stat = 0.274516    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.904595
 Kendall's correlation score: 0.693759
 Spearman's correlation score: 0.869866
 Kolmogorov-Smirnov test:    KS-stat = 0.061679    p-value = 1.060e-253


 Class: identity_hate
 Pearson's correlation score: 0.829362
 Kendall's correlation score: 0.630277
 Spearman's correlation score: 0.825395
 Kolmogorov-Smirnov test:    KS-stat = 0.137049    p-value = 0.000e+00

Corelation between :  pooled-gru-with-preprocessing.csv

 Kendall's correlation score: 0.634540
 Spearman's correlation score: 0.833599
 Kolmogorov-Smirnov test:    KS-stat = 0.426327    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.940026
 Kendall's correlation score: 0.726430
 Spearman's correlation score: 0.893693
 Kolmogorov-Smirnov test:    KS-stat = 0.376054    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.878359
 Kendall's correlation score: 0.685684
 Spearman's correlation score: 0.870902
 Kolmogorov-Smirnov test:    KS-stat = 0.397091    p-value = 0.000e+00

Corelation between :  toxic-avenger.9823.csv hight_of_blending.9835.csv

 Class: toxic
 Pearson's correlation score: 0.991789
 Kendall's correlation score: 0.861469
 Spearman's correlation score: 0.964793
 Kolmogorov-Smirnov test:    KS-stat = 0.052767    p-value = 8.501e-186


 Class: severe_toxic
 Pearson's correlation score: 0.958888
 Kendall's correlation score: 0.859388
 Spearman's correlation score: 0.960870
 Kolmogorov-

 Kolmogorov-Smirnov test:    KS-stat = 0.991195    p-value = 0.000e+00

Corelation between :  toxic-avenger.9823.csv lvl0_wordbatch_clean_sub.9805.csv

 Class: toxic
 Pearson's correlation score: 0.916967
 Kendall's correlation score: 0.718307
 Spearman's correlation score: 0.888950
 Kolmogorov-Smirnov test:    KS-stat = 0.782978    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.870725
 Kendall's correlation score: 0.337838
 Spearman's correlation score: 0.478214
 Kolmogorov-Smirnov test:    KS-stat = 0.994561    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.925075
 Kendall's correlation score: 0.621792
 Spearman's correlation score: 0.799131
 Kolmogorov-Smirnov test:    KS-stat = 0.882799    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.727148
 Kendall's correlation score: 0.244180
 Spearman's correlation score: 0.354789
 Kolmogorov-Smirnov test:    KS-stat = 0.997578    p-value = 0.000e+00


 Class: insult
 Pe

 Spearman's correlation score: 0.739622
 Kolmogorov-Smirnov test:    KS-stat = 0.295350    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.954793
 Kendall's correlation score: 0.675587
 Spearman's correlation score: 0.847309
 Kolmogorov-Smirnov test:    KS-stat = 0.439659    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.832806
 Kendall's correlation score: 0.521816
 Spearman's correlation score: 0.718180
 Kolmogorov-Smirnov test:    KS-stat = 0.394571    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.922171
 Kendall's correlation score: 0.674125
 Spearman's correlation score: 0.850705
 Kolmogorov-Smirnov test:    KS-stat = 0.300469    p-value = 0.000e+00


 Class: identity_hate
 Pearson's correlation score: 0.872690
 Kendall's correlation score: 0.603542
 Spearman's correlation score: 0.797084
 Kolmogorov-Smirnov test:    KS-stat = 0.319383    p-value = 0.000e+00



In [57]:
# compare one against all

first_file = "blend_it_all.9867.csv"

for f in submissions:
    if f != first_file:
        print("Corelation between : ", f, first_file)
        corr(fold1+f, fold1+first_file)

Corelation between :  blend_it_all.gru_glove.fasttext.9868.csv blend_it_all.9867.csv

 Class: toxic
 Pearson's correlation score: 0.999493
 Kendall's correlation score: 0.982305
 Spearman's correlation score: 0.999358
 Kolmogorov-Smirnov test:    KS-stat = 0.381069    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.999211
 Kendall's correlation score: 0.983079
 Spearman's correlation score: 0.999126
 Kolmogorov-Smirnov test:    KS-stat = 0.747774    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.999562
 Kendall's correlation score: 0.981105
 Spearman's correlation score: 0.999171
 Kolmogorov-Smirnov test:    KS-stat = 0.560001    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.998590
 Kendall's correlation score: 0.968439
 Spearman's correlation score: 0.997435
 Kolmogorov-Smirnov test:    KS-stat = 0.862618    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.999571
 Kendall's correlation score: 


 Class: toxic
 Pearson's correlation score: 0.971193
 Kendall's correlation score: 0.807253
 Spearman's correlation score: 0.943038
 Kolmogorov-Smirnov test:    KS-stat = 0.562965    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.949061
 Kendall's correlation score: 0.628200
 Spearman's correlation score: 0.804900
 Kolmogorov-Smirnov test:    KS-stat = 0.860979    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.982096
 Kendall's correlation score: 0.746314
 Spearman's correlation score: 0.898469
 Kolmogorov-Smirnov test:    KS-stat = 0.711773    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.882900
 Kendall's correlation score: 0.590806
 Spearman's correlation score: 0.783680
 Kolmogorov-Smirnov test:    KS-stat = 0.982783    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.974400
 Kendall's correlation score: 0.750332
 Spearman's correlation score: 0.903976
 Kolmogorov-Smirnov test:    KS-stat


 Class: toxic
 Pearson's correlation score: 0.976665
 Kendall's correlation score: 0.758116
 Spearman's correlation score: 0.910949
 Kolmogorov-Smirnov test:    KS-stat = 0.655552    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.944337
 Kendall's correlation score: 0.635690
 Spearman's correlation score: 0.813180
 Kolmogorov-Smirnov test:    KS-stat = 0.910997    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.982298
 Kendall's correlation score: 0.722513
 Spearman's correlation score: 0.880787
 Kolmogorov-Smirnov test:    KS-stat = 0.733547    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.904344
 Kendall's correlation score: 0.560102
 Spearman's correlation score: 0.752925
 Kolmogorov-Smirnov test:    KS-stat = 0.973512    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.973547
 Kendall's correlation score: 0.737245
 Spearman's correlation score: 0.895767
 Kolmogorov-Smirnov test:    KS-stat

In [ ]:
blend_it_all.9867.csv
-------

gru_glove_128_95p_reg.10fold_avg.9853.csv
gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csvf
lstm_l1_256_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep2_batch_128_nadam_9834
lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv
lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv
cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv
toxic_lr_char_tdidf.9792.csv - 1


In [ ]:
# https://www.kaggle.com/peterhurford/lgb-gru-lr-lstm-nb-svm-average-ensemble/code

import numpy as np
import pandas as pd


gru = pd.read_csv('../input/pooled-gru-fasttext/submission.csv') # PL score 0.9829
lstm_nb_svm = pd.read_csv('../input/minimal-lstm-nb-svm-baseline-ensemble/submission.csv') # 0.9811
lr = pd.read_csv('../input/logistic-regression-with-words-and-char-n-grams/submission.csv') # 0.9788
lgb = pd.read_csv('../input/lightgbm-with-select-k-best-on-tfidf/lgb_submission.csv') # 0.9785


# The value of an ensemble is (a) the individual scores of the models and
# (b) their correlation with one another. We want multiple individually high
# scoring models that all have low correlations. Based on this analysis, it
# looks like these kernels have relatively low correlations and will blend to a
# much higher score.
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
for label in labels:
    print(label)
    print(np.corrcoef([lgb[label].rank(pct=True), gru[label].rank(pct=True), lr[label].rank(pct=True), lstm_nb_svm[label].rank(pct=True)]))

submission = pd.DataFrame()
submission['id'] = lgb['id']
for label in labels:
    submission[label] = lgb[label].rank(pct=True) * 0.15 + gru[label].rank(pct=True) * 0.4 + lr[label].rank(pct=True) * 0.15 + lstm_nb_svm[label].rank(pct=True) * 0.3

submission.to_csv('submission.csv', index=False)

### Blending

In [70]:
fold1 = '../submissions/ensemble/'
fold2 = '../submissions/ensemble/downloads/'

# https://www.kaggle.com/codename007/toxic-avenger-spin-lb-0-9832
ave = pd.read_csv(fold2+"toxic-avenger.9823.csv")

# https://www.kaggle.com/tunguz/blend-of-blends-1/output 
supbl= pd.read_csv(fold2+'superblend_1.csv')

# https://www.kaggle.com/nuhsikander/blend-of-blends-can-be-toxic-lb-0-9837 0.9837
best = pd.read_csv(fold2+'hight_of_blending.9835.csv')

# https://www.kaggle.com/ogrellier/wordbatch-fm-ftrl-using-mse-lb-0-9804 0.9805 FM FLTR
wordbtch = pd.read_csv(fold2+'lvl0_wordbatch_clean_sub.9805.csv')

# https://www.kaggle.com/hhstrand/oof-stacking-regime 0.9858
oofs = pd.read_csv(fold2+'oof-stacking-regime.9858.csv')

# https://www.kaggle.com/antmarakis/another-blend-tinkered-by-correlation - 0.9855
corrbl = pd.read_csv(fold2+'corr_blend.9855.csv')

# GRU pooling with R - https://www.kaggle.com/krosam/why-a-such-low-score-with-r-and-keras 0.9835
rkera = pd.read_csv(fold2+'pooled_gru_glove_r.9815.csv')

# https://www.kaggle.com/ogrellier/lgbm-with-words-and-chars-n-gram 0.9792
lgbm = pd.read_csv(fold2+'lvl0_lgbm_clean_sub.9792.csv')

# https://www.kaggle.com/kailex/tidy-xgboost-glmnet-text2vec-lsa 0.9786
tidy = pd.read_csv(fold2+'tidy_xgb_glm.9784.csv')

# https://www.kaggle.com/eashish/bidirectional-gru-with-convolution 0.9850
grucnn = pd.read_csv(fold2+'bi-gru-cnn.9850.csv')

# https://www.kaggle.com/eashish/bidirectional-gru-with-convolution
bilst = pd.read_csv(fold2+'bi-lstm-cnn.9841.csv')


# https://www.kaggle.com/prashantkikani/pooled-gru-glove-with-preprocessing 0.9823
gruglo = pd.read_csv(fold2+"pooled-gru-with-preprocessing.9823.csv")
# BL version
gruglo_bl = pd.read_csv(fold1+"gru_glove_128_95p_reg.10fold_avg.9853.csv")

# https://www.kaggle.com/zhbain/pooled-gru-fasttext-6c07c9/code #0.9833
fast = pd.read_csv(fold2+'pooled-gru-fasttext.9833.csv')
# BL version
fast_bl = pd.read_csv(fold1+'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv')

dpcnn = pd.read_csv(fold1+'dpcnn_glove_32_95p_reg.20fold_avg.9835.csv')

##
#gru_glove_128_95p_reg.10fold_avg.9853.csv
#lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv

#gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv
#lstm_l1_256_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep2_batch_128_nadam_9834

#lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv
#cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv
#toxic_lr_char_tdidf.9792.csv - 1

In [69]:
b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (fast[i] * 2 + gruglo[i] * 2 + grucnn[i] * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + gruglo_bl[i] * 4 + fast_bl[i] * 2 + dpcnn[i] * 4) /  43
    
b1.to_csv('../submissions/blend_it_all.gru_glove.fasttext.dpcnn.csv', index = False)

In [63]:
# experiments

b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (fast[i] * 2 + grucnn[i] * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + gruglo_bl[i] * 5 + fast_bl[i] * 3) /  39
    
b1.to_csv('../submissions/blend_it_all.9853.9843.csv', index = False)

* https://www.kaggle.com/tilii7/ensemble-weights-minimization-vs-mcmc
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/51546
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50913
* https://www.kaggle.com/peterhurford/lgb-gru-lr-lstm-nb-svm-average-ensemble/output
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50827
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/51274